Tools for Analyzing Grid Connectivity
======

Towards the development of tools for improving grid stability using grid topology changes
-------

In this notebook, we will preview some of the tools that I developed to study grid topologies, with an eye on trying to develop tools that can be used to make educated changes to the grid topology in certain circumstances (e.g. to cure line overloads).

The tools are developed using the `rte_case14_realistic` environment, and using the `grid2op` framework as a jumping-off point. However, this is initially only used to load the case14 scenario (and for the plotting capabilities); in principle these tools are not strictly dependent on the `grid2op` environment.

The code also includes a small histogramming class that I developed to store histogram-like data (without the need to keep around large lists of the data itsef.)

In [ ]:
%load_ext autoreload
import os
import sys
import grid2op
import matplotlib.pyplot as plt
import numpy as np
import itertools
import copy

# These helper modules are mine:
%aimport CommonHelpers
%aimport BusTopologyHelpers
%aimport Substation
%aimport TopologyHelpers
%autoreload

# This is a small histogramming class -- see https://github.com/brendlin/matplotlibHistos
sys.path.insert(0, "../")
from matplotlibHistos.Histo import Histo

In [ ]:
env = grid2op.make(test=True)

In [ ]:
from grid2op.PlotGrid import PlotMatplot
plot_helper = PlotMatplot(env.observation_space)
_ = plot_helper.plot_info(line_values=['line %d'%(el) for el in range(env.n_line)])

How is the grid topology represented in `grid2op`?
------
Let's explore the `grid2op` environment enough to understand how it encodes the grid connectivity. A few summary values describe the basics of the grid:

In [ ]:
print('dim topo: env.dim_topo (2*n_line + n_load + n_gen): {:d} ({:d})'.format(env.dim_topo,env.n_line*2 + env.n_load + env.n_gen))
print('Number of lines, env.n_line:',env.n_line)
print('Number of loads, env.n_load:',env.n_load)
print('Number of loads, env.n_gen:',env.n_gen)
print('Number of connections to each sub env.sub_info: ',env.sub_info)

Additional properties detail how elements are connected to one another:

In [ ]:
print('Line origin    to subid env.line_or_to_subid:',env.line_or_to_subid)
print('Line extremity to subid env.line_ex_to_subid:',env.line_ex_to_subid)
print('Generator      to subid env.gen_to_subid    :',env.gen_to_subid)
print('Load           to subid env.load_to_subid   :',env.load_to_subid)
print()
print('Load-to-sub           position env.load_to_sub_pos   : ',env.load_to_sub_pos)
print('Generator-to-sub      position env.gen_to_sub_pos    : ',env.gen_to_sub_pos)
print('Line origin-to-sub    position env.line_or_to_sub_pos: ',env.line_or_to_sub_pos)
print('Line extremity-to-sub position env.line_ex_to_sub_pos: ',env.line_ex_to_sub_pos)

Other properties include the `topo vect` and the `grid object types`, which will tell you the ID of the object connected to a substation. They are shown here for completeness, but so far I have not used them.

In [ ]:
print('Topo vec (len {:d}): '.format(len(env.backend.get_topo_vect())),list(a for a in env.backend.get_topo_vect()))
print()
print('env.grid_objects_types:\n',env.grid_objects_types[:10],'...')
print('This will tell you the ID of the object connected to which bus.')
print('The order is [sub_id,load_id,gen_id,origin_id,extremity_id].')

Adjacency Matrix Representation of the Grid
--------
Now we try to represent the grid as a **graph**, using the buses and generators/loads as vertices. One vertex corresponds to one substation bus (either bus 1 or bus 2), one load or one generator. If a line connects e.g. a substation bus $i$ to generator $j$, then the entry $a_{ij}$ (and $a_{ji}$) is filled with a 1.

In [ ]:
adjacency_matrix = TopologyHelpers.MakeAdjacencyMatrix(env,n_buses=1,skipExternals=False)

print('Adjacency matrix (columns are labeled as substation, generator or load):')
print(' '.join('S' for a in env.sub_info),' '.join('g' for a in range(env.n_gen)),' '.join('L' for a in range(env.n_load)))
print('--'*(len(env.sub_info) + env.n_gen + env.n_load - 1))
for i in adjacency_matrix :        
    print(' '.join(('%d'%(a) if a>0 else '·') for a in list(i)))

A related representation is the **Laplacian matrix**, in which the diagonal element is set to the negative sum of elements in a row or column:

In [ ]:
laplacian_matrix = TopologyHelpers.MakeLaplacian(env,n_buses=1,skipExternals=True)
print('\nLaplacian matrix:')
for i in laplacian_matrix :
    tmp = ''
    for j in i :
        tmp += '{:>3}'.format(-j)
    print(tmp)

For clarity, we can print the line IDs of the non-zero elements of this matrix to understand their role in this representation:

In [ ]:
TopologyHelpers.PrintLineIDs(env)